In [2]:
import numpy as np
import pandas as pd
import xgboost

df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [3]:
X=df.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]

In [4]:
X.head()

,Reco_Policy_Premium,Region_Code,Reco_Policy_Cat,Upper_Age,Holding_Policy_Duration,Reco_Insurance_Type
0,11628.0,3213,22,36,14+,Individual
1,30510.0,1117,22,75,NaN,Joint
2,7450.0,3732,19,32,1.0,Individual
3,17780.0,4378,19,52,14+,Joint
4,10404.0,2190,16,44,3.0,Individual


In [5]:
X_Sub=df_test.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]

In [6]:
y=df.iloc[:,-1]

In [7]:
cat=['Reco_Insurance_Type']
num=[i for i in X.columns if i not in cat]

In [8]:
X1=X.loc[:,cat]
X2=X.loc[:,num]

X1_Sub=X_Sub.loc[:,cat]
X2_Sub=X_Sub.loc[:,num]

In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X1=X1.apply(le.fit_transform)

In [10]:
X1_Sub=X1_Sub.apply(LabelEncoder().fit_transform)

In [11]:
X2.nunique()

Reco_Policy_Premium        6977
Region_Code                5316
Reco_Policy_Cat              22
Upper_Age                    58
Holding_Policy_Duration      15
dtype: int64

In [12]:
unique_items2=X2['Holding_Policy_Duration'].unique()
map_dict2={}
for i,j in enumerate(unique_items2):
    map_dict2[j]=i
map_dict2[np.nan]=np.nan
X2['Holding_Policy_Duration']=X2['Holding_Policy_Duration'].map(map_dict2)

In [13]:
X2.head()

,Reco_Policy_Premium,Region_Code,Reco_Policy_Cat,Upper_Age,Holding_Policy_Duration
0,11628.0,3213,22,36,0.0
1,30510.0,1117,22,75,NaN
2,7450.0,3732,19,32,2.0
3,17780.0,4378,19,52,0.0
4,10404.0,2190,16,44,3.0


In [14]:
X2_Sub['Holding_Policy_Duration']=X2_Sub['Holding_Policy_Duration'].map(map_dict2)

In [15]:
X=pd.concat([X1,X2],axis=1)
X_Sub=pd.concat([X1_Sub,X2_Sub],axis=1)

In [16]:
X.head()

,Reco_Insurance_Type,Reco_Policy_Premium,Region_Code,Reco_Policy_Cat,Upper_Age,Holding_Policy_Duration
0,0,11628.0,3213,22,36,0.0
1,1,30510.0,1117,22,75,NaN
2,0,7450.0,3732,19,32,2.0
3,1,17780.0,4378,19,52,0.0
4,0,10404.0,2190,16,44,3.0


In [17]:
X_Sub.head()

,Reco_Insurance_Type,Reco_Policy_Premium,Region_Code,Reco_Policy_Cat,Upper_Age,Holding_Policy_Duration
0,0,11934.0,156,5,30,12.0
1,1,32204.8,7,18,69,3.0
2,0,9240.0,564,17,28,8.0
3,0,9086.0,1177,18,23,3.0
4,0,22534.0,951,5,75,NaN


In [18]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Reco_Insurance_Type      50882 non-null  int32  
 1   Reco_Policy_Premium      50882 non-null  float64
 2   Region_Code              50882 non-null  int64  
 3   Reco_Policy_Cat          50882 non-null  int64  
 4   Upper_Age                50882 non-null  int64  
 5   Holding_Policy_Duration  30631 non-null  float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 2.1 MB


In [19]:
X_Sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21805 entries, 0 to 21804
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Reco_Insurance_Type      21805 non-null  int32  
 1   Reco_Policy_Premium      21805 non-null  float64
 2   Region_Code              21805 non-null  int64  
 3   Reco_Policy_Cat          21805 non-null  int64  
 4   Upper_Age                21805 non-null  int64  
 5   Holding_Policy_Duration  13202 non-null  float64
dtypes: float64(2), int32(1), int64(3)
memory usage: 937.1 KB


In [20]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=pd.DataFrame(sc.fit_transform(X),columns=X.columns)

In [21]:
X_Sub=pd.DataFrame(sc.transform(X_Sub),columns=X_Sub.columns)

In [22]:
X.head()

,Reco_Insurance_Type,Reco_Policy_Premium,Region_Code,Reco_Policy_Cat,Upper_Age,Holding_Policy_Duration
0,-0.505203,-0.387852,1.039425,1.085830,-0.511625,-1.338767
1,1.979403,2.477394,-0.432415,1.085830,1.741395,NaN
2,-0.505203,-1.021842,1.403874,0.612688,-0.742704,-0.893719
3,1.979403,0.545682,1.857504,0.612688,0.412691,-1.338767
4,-0.505203,-0.573588,0.321060,0.139547,-0.049467,-0.671196


In [23]:
from sklearn.impute import KNNImputer
imputer=KNNImputer(n_neighbors=5)
X=pd.DataFrame(imputer.fit_transform(X),columns=X.columns)

In [24]:
X_Sub=pd.DataFrame(imputer.transform(X_Sub),columns=X_Sub.columns)

In [25]:
X.head()

,Reco_Insurance_Type,Reco_Policy_Premium,Region_Code,Reco_Policy_Cat,Upper_Age,Holding_Policy_Duration
0,-0.505203,-0.387852,1.039425,1.085830,-0.511625,-1.338767
1,1.979403,2.477394,-0.432415,1.085830,1.741395,0.129889
2,-0.505203,-1.021842,1.403874,0.612688,-0.742704,-0.893719
3,1.979403,0.545682,1.857504,0.612688,0.412691,-1.338767
4,-0.505203,-0.573588,0.321060,0.139547,-0.049467,-0.671196


In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [28]:
clf = xgboost.XGBClassifier()
clf.fit(X,y)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:41:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
y_pred=clf.predict(X_test)

In [31]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test, y_pred)

0.5484659497197119

In [32]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(Y_test, y_pred)

0.7739019357374471

In [33]:
confusion_matrix(Y_test, y_pred)

array([[7619,   39],
       [2262,  257]], dtype=int64)

In [34]:
Y_Sub=clf.predict(X_Sub)

In [35]:
id=pd.DataFrame(df_test.ID,columns=['ID'])
respo=pd.DataFrame(Y_Sub,columns=['Response'])
pred=pd.concat([id,respo],axis=1)

In [36]:
pred.head()

,ID,Response
0,50883,0
1,50884,0
2,50885,0
3,50886,0
4,50887,0


In [37]:
filename='Submission7.csv'
pred.to_csv(filename,index=False)